In [88]:
import pandas as pd
import numpy as np
import re
import spacy
import ru_core_news_lg
nlp = ru_core_news_lg.load()

In [91]:
def make_clean(txt):
    txt = txt.lower()
    
    pattern = re.compile(r'[\d$#№!?,.@_:;()\+\*"%\\/-]')
    txt = re.sub(pattern, ' ', txt)
    
    pattern_eng = re.compile('[a-z]')
    txt = re.sub(pattern_eng, ' ', txt)    
    
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')       
    txt = txt.replace('\t',' ')
    
    while '  ' in txt:
        txt = txt.replace('  ',' ')
        
    txt = txt.rstrip()
    txt = txt.lstrip()
        
    return txt

In [93]:
# Word2Vec

In [94]:
import gensim
gensim.utils.simple_preprocess(transacts['v_payment_desc'][100])

['личный', 'средства', 'предпринимателя', 'без', 'ндс']

In [95]:
#transacts.v_payment_desc.apply(gensim.utils.simple_preprocess)

In [96]:
full_list = []
for i in range(len(transacts)):
    full_list.append([token.lemma_ for token in nlp(transacts['full_description'][i]) if not token.is_stop and len(token.lemma_)>3]) 

In [97]:
print(len(full_list))

1074286


In [98]:
import gensim.models

In [99]:
model = gensim.models.Word2Vec(sentences=full_list, min_count=2, window=3, vector_size=100, sg=1, seed=42, compute_loss=True)

In [100]:
model.corpus_count

1074286

In [101]:
model.wv['погашение']

array([ 0.16364707,  0.26361158, -0.22940241, -0.41586262,  0.39979815,
        0.08066381,  0.3106773 ,  0.7670577 , -0.4613791 , -0.09823401,
        0.70756084,  0.19681834,  0.5485931 , -0.2562328 , -0.12064777,
       -0.44429082,  0.639027  , -0.22210835, -0.11405789, -0.16706844,
        0.5023008 , -0.07339681,  0.4396937 ,  0.8064233 ,  0.5021427 ,
        0.9570349 ,  0.37244862,  0.74582034, -0.7722286 , -0.96683127,
        0.15446855,  0.15104489, -1.0382084 ,  0.09913738,  0.39783874,
       -0.09797168,  0.9587439 , -0.38797367, -0.11883784,  0.71954495,
       -0.18509045, -0.41101345, -0.6773648 , -0.63847566,  1.3858399 ,
       -0.68820626, -0.01351582,  0.8965673 , -0.06887279,  0.24098001,
       -0.90315866, -0.73267025,  0.76434803, -0.22466543, -0.01956002,
       -0.07030944,  0.60606587, -0.5634238 , -0.48285475, -0.19561243,
        0.7169033 ,  0.4050764 , -0.80906034,  0.45216104,  0.4840323 ,
        0.30793086, -0.4813744 , -1.0406488 ,  0.75555736,  0.26

In [102]:
model.wv.most_similar('зарплата')

[('отпускные', 0.6796675324440002),
 ('махамат', 0.657871663570404),
 ('половина', 0.6507164239883423),
 ('отпускных', 0.6471732258796692),
 ('мусса', 0.6433866620063782),
 ('тахир', 0.6424691677093506),
 ('федякин', 0.6381070613861084),
 ('аморозо', 0.6332544088363647),
 ('полностью', 0.6307960152626038),
 ('отпускной', 0.6301730871200562)]

In [103]:
model.wv.similarity(w1='средство', w2='личный')

0.55344003

In [104]:
from gensim.models import KeyedVectors

word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')
vector = wv['погашение']

In [105]:
vector

array([ 0.16364707,  0.26361158, -0.22940241, -0.41586262,  0.39979815,
        0.08066381,  0.3106773 ,  0.7670577 , -0.4613791 , -0.09823401,
        0.70756084,  0.19681834,  0.5485931 , -0.2562328 , -0.12064777,
       -0.44429082,  0.639027  , -0.22210835, -0.11405789, -0.16706844,
        0.5023008 , -0.07339681,  0.4396937 ,  0.8064233 ,  0.5021427 ,
        0.9570349 ,  0.37244862,  0.74582034, -0.7722286 , -0.96683127,
        0.15446855,  0.15104489, -1.0382084 ,  0.09913738,  0.39783874,
       -0.09797168,  0.9587439 , -0.38797367, -0.11883784,  0.71954495,
       -0.18509045, -0.41101345, -0.6773648 , -0.63847566,  1.3858399 ,
       -0.68820626, -0.01351582,  0.8965673 , -0.06887279,  0.24098001,
       -0.90315866, -0.73267025,  0.76434803, -0.22466543, -0.01956002,
       -0.07030944,  0.60606587, -0.5634238 , -0.48285475, -0.19561243,
        0.7169033 ,  0.4050764 , -0.80906034,  0.45216104,  0.4840323 ,
        0.30793086, -0.4813744 , -1.0406488 ,  0.75555736,  0.26

In [106]:
#GloVe

In [87]:
#!pip install glove_python

In [ ]:
from glove import Corpus, Glove


corpus = Corpus()
corpus.fit(full_list, window=10)

glove = Glove(learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30)
glove.add_dictionary(corpus.dictionary)

In [107]:
txt = nlp(transacts['full_description'][10])

for token in nlp(txt):
    print(token, "vector: ", token.has_vector, "OOV: ", token.is_oov)

дебетовые vector:  False OOV:  True
операции vector:  True OOV:  False
по vector:  True OOV:  False
пластиковым vector:  True OOV:  False
картам vector:  True OOV:  False
мир vector:  True OOV:  False
клиентов vector:  True OOV:  False
банка vector:  True OOV:  False
в vector:  True OOV:  False
устройствах vector:  True OOV:  False
сторонних vector:  True OOV:  False
банков vector:  True OOV:  False


In [119]:
base_token = nlp("банк")

for token in txt:
    if token.has_vector:
        print(token, ' | ', token.similarity(base_token))

операции  |  0.25299376946849433
по  |  0.10214186284249434
пластиковым  |  -0.024998980841946745
картам  |  0.12871489737170375
мир  |  0.09037674365518401
клиентов  |  0.33366377791687085
банка  |  0.599426025213813
в  |  0.17035344045835563
устройствах  |  -0.013292893457194085
сторонних  |  0.09605180191065005
банков  |  0.5947529985119299


In [77]:
#FastText

In [109]:
#!pip install fasttext
#import fasttext

Looking in indexes: https://artifactory.intranet/artifactory/api/pypi/pypi/simple


In [110]:
def make_lemma(txt):
    
    txt = nlp(txt)
    txt_new = [token.lemma_ for token in txt if not token.is_stop and not token.is_punct and len(token)>3]
    txt = ' '.join(txt_new)
    return txt

In [111]:
transacts['lemma_description'] = transacts['full_description'].apply(lambda x: make_lemma(x))

In [112]:
transacts['lemma_description'].to_csv('transacts.txt', header=False, index=False, sep='\t')

In [113]:
#векторное представление слов
model = fasttext.train_unsupervised(input='transacts.txt', epoch=20, model='skipgram')

Read 8M words
Number of words:  7003
Number of labels: 0
Progress: 100.0% words/sec/thread:   72885 lr:  0.000000 avg.loss:  0.923903 ETA:   0h 0m 0s


In [114]:
model.get_nearest_neighbors("банк")

[(0.8517051339149475, 'устройство'),
 (0.8508301973342896, 'сторонний'),
 (0.8411794304847717, 'дебетовый'),
 (0.8209928870201111, 'кредитовый'),
 (0.8138197064399719, 'пластиковый'),
 (0.7710592746734619, 'клиент'),
 (0.7244216799736023, 'карта'),
 (0.7179813385009766, 'операция'),
 (0.6684747338294983, 'зачёт'),
 (0.6643556356430054, 'нбанк')]

In [118]:
#векторное представление слов c ngramms
model = fasttext.train_unsupervised(input='transacts.txt', epoch=20, minn=2, maxn=5)

Read 8M words
Number of words:  7003
Number of labels: 0
Progress: 100.0% words/sec/thread:   59581 lr:  0.000000 avg.loss:  0.950857 ETA:   0h 0m 0s 86.7% words/sec/thread:   62242 lr:  0.006667 avg.loss:  0.832415 ETA:   0h 0m 4s 0s


In [117]:
model.get_nearest_neighbors("банк")

[(0.8055684566497803, 'баня'),
 (0.7657368183135986, 'банк'),
 (0.7204897999763489, 'дебетовый'),
 (0.7090697884559631, 'сторонний'),
 (0.7076113820075989, 'терминал'),
 (0.7041202783584595, 'кредитовый'),
 (0.6998031139373779, 'банка'),
 (0.6928038001060486, 'устройство'),
 (0.6862907409667969, 'штуцер'),
 (0.6742855310440063, 'агатовый')]